In [ ]:
%pylab inline

In [ ]:
def waveFunc(M, x0):
    n = arange(M)
    hbar = 2*pi/M
    x = 2*pi*n/M
    psi0 = zeros(M, dtype=complex)
    psi0[:] = exp(-(x-x0)**2/2/hbar)
    psi0 = psi0/np.linalg.norm(psi0)
    return x, psi0

In [ ]:
x, psi0 = waveFunc(100,pi)

In [ ]:
plot(x,psi0)
show()
plot(x, abs(psi0)**2)
show()

In [ ]:
def gaussianWavePacket(M, x0, p0):
    n = arange(M)
    x = 2*pi*n/M
    p = 2*pi*n/M
    hbar = 2*pi/M
    
    n0 = x0*M/2/pi
    m0 = p0*M/2/pi
    
    psi_exp = 1j*2*pi*m0*n/M
    psil = exp(psi_exp)
    nt, l = meshgrid(n, arange(-4,5))
    psi_exp = -pi/M*(nt-n0+l*M)**2
    psir = sum(exp(psi_exp), axis=0)

    N = np.linalg.norm(psil*psir)
    psi_x = psil*psir/N
    
    psi_p = 1/sqrt(M)*fft.fft(psi_x)
    return x, p, psi_x, psi_p

def phaseVectors(x, p, K):
    hbar = 2*pi/x.size
    V = exp( -1j/hbar * K * cos(x) )
    P = exp(-1j/(2*hbar) * p**2)
    return V, P

def evolveOneStep(V, P, psi_x):
    M = psi_x.size
    fm = V*psi_x
    sm = P*fft.fft(fm)
    s = fft.ifft(sm)
    return s

def StdMap(n, K, x0 = 1., p0 = 1.):
    x = zeros(n); p = zeros(n)
    x[0] = x0; p[0] = p0
    for i in arange(n-1):
        p[i+1] = mod(p[i] + K*sin(x[i]),2*pi)
        x[i+1] = mod(x[i] + p[i+1],2*pi)
    return x,p

In [ ]:
M = 1000
K = 1.1
x0 = 1
p0 = 0.1
x, p, psi_x, psi_p = gaussianWavePacket(M, x0, p0)

fig=figure(figsize=(10,6))
plot(x, abs(psi_x)**2, label="$|\psi(x_n)|^2$", lw=2)
axvline(x0, color="k", label="$x_0$")
xlim(0,2*pi)
legend(fontsize=16)
savefig("GWP_psi_x.pdf")
show()

fig=figure(figsize=(10,6))
plot(p,abs(psi_p)**2, label="$|\psi(p_n)|^2$", lw=2)
axvline(p0, color="k", label="$p_0$")
xlim(0,2*pi)
legend(fontsize=16)
savefig("GWP_psi_n.pdf")
show()

#V, P = phaseVectors(x, p, K)
#psi_nx = evolveOneStep(V, P, psi_x)
#plot(abs(psi_nx)**2)
#show()
#sum(abs(psi_p)**2)

In [ ]:
M = 1000
K = 1.1
x0 = 1
p0 = 0.1
kicks = 50

std_x, std_p = StdMap(kicks+1, K, x0, p0)

x, p, psi_x, psi_p = gaussianWavePacket(M, x0, p0)
V, P = phaseVectors(x, p, K)

fig = figure(figsize=(10,6))
xlabel(r"$x_n$", fontsize=18)
ylabel("$|\psi(x_n)|^2$", fontsize=18)
xlim(0,2*pi)
kick_text = fig.text(0.145, 0.82, "Kick = 0", fontsize=14)
plot(x, abs(psi_x)**2, lw=2)
axvline(std_x[0], color="k", label="$x_{n,Std}$")
#axvline(x0, color="k", label="$x_0$")
title("$x_0$ = " + str(x0) + ", $p_0$ = " + str(p0) + ", K = " +str(K), fontsize=20)
#title("$x_0$ = $\pi$, $p_0$ = $\pi$, K = " +str(K), fontsize=20)
savefig("frames/" + str(0).zfill(4) + ".pdf")
show()

for i in arange(kicks):
    
    psi_x = evolveOneStep(V, P, psi_x)

    fig = figure(figsize=(10,6))
    xlabel(r"$x_n$", fontsize=18)
    ylabel("$|\psi(x_n)|^2$", fontsize=18)
    xlim(0,2*pi)
    kick_text = fig.text(0.145, 0.82, "Kick = " + str(i+1), fontsize=14)
    plot(x, abs(psi_x)**2, lw=2)
    title("$x_0$ = " + str(x0) + ", $p_0$ = " + str(p0) + ", K = " +str(K), fontsize=20)
    axvline(std_x[i+1], color="k", label="$x_{n,Std}$")
    #title("$x_0$ = $\pi$, $p_0$ = $\pi$, K = " +str(K), fontsize=20)
    savefig("frames/" + str(i+1).zfill(4) + ".pdf")
    plt.close()
    
#show()
#sum(abs(psi_x)**2)

In [ ]:
M = 100
K = 0.6
kicks = 50

x0 = 1.5
p0 = 0.3

std_x, std_p = StdMap(kicks+1, K, x0, p0)

x, p, psi_x, psi_p = gaussianWavePacket(M, x0, p0)
V, P = phaseVectors(x, p, K)

Qdist = zeros((M,M), dtype=complex)

for i in arange(M):
    x0 = 2*pi*i/M
    x, p, psiG_n0, psi_p = gaussianWavePacket(M, x0, 0)
    Qdist[:,i] = 1./sqrt(M)*fft.fft(np.conj(psiG_n0)*psi_x)
    
n = arange(M)
X, Y = meshgrid(2*pi*n/M,2*pi*n/M)
pcolor(X, Y, abs(Qdist))
colorbar(format="%.4f")
xlabel("$x_n$", fontsize=16)
ylabel("$p_n$", fontsize=16)
title(r"$|Q(1.5,0.3)|^2$, K=" + str(K), fontsize=18)
text(4.9,5.6, "Kick 0", color="w", fontsize=14)
plot(std_x[0], std_p[0], "r.")
xlim(0,6.22035345)
ylim(0,6.22035345)
tight_layout()
savefig("frames/" + str(0).zfill(4) + ".pdf")
#savefig("Husimi_initial.pdf")
show()
#print sum(1./M*abs(Qdist)**2)

for k in arange(kicks):
    psi_x = evolveOneStep(V, P, psi_x)
    
    for i in arange(M):
        x0 = 2*pi*i/M
        x, p, psiG_n0, psi_p = gaussianWavePacket(M, x0, 0)
        Qdist[:,i] = 1./sqrt(M)*fft.fft(np.conj(psiG_n0)*psi_x)
        
    
    n = arange(M)
    X, Y = meshgrid(2*pi*n/M,2*pi*n/M)
    pcolor(X, Y, abs(Qdist)**2)
    colorbar(format="%.4f")
    xlabel("$x_n$", fontsize=16)
    ylabel("$p_n$", fontsize=16)
    title(r"$|Q(1.5,0.3)|^2$, K=" + str(K), fontsize=18)
    text(4.9,5.6, "Kick " + str(k+1), color="w", fontsize=14)
    plot(std_x[k+1], std_p[k+1], "r.")
    xlim(0,6.22035345)
    ylim(0,6.22035345)
    tight_layout()
    savefig("frames/" + str(k+1).zfill(4) + ".pdf")
    plt.close()

#show()
#print sum(1./M*abs(Qdist)**2)

In [ ]:
for i in arange(M):
    x0 = 2*pi*i/M
    x, p, psiG_n0, psi_p = gaussianWavePacket(M, x0, 0)
    Qdist[:,i] = 1./sqrt(M)*fft.fft(np.conj(psiG_n0)*psi_x)
pcolor(X, Y, abs(Qdist)**2)
colorbar()
show()
sum(abs(Qdist)**2)

In [ ]:
n = arange(20)
l = arange(-4,5)
nt = zeros(20)
for i in arange(20):
    nt[i] = sum(-pi/M*(n[i]-n0+l*M)**2)